# Tensorflow Low Level APIs

https://www.tensorflow.org/guide/low_level_intro

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

You might think of TensorFlow Core programs as consisting of two discrete sections:   
1- Building the computational graph (a tf.Graph).  
2 -Running the computational graph (using a tf.Session).  

### Graph

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

In [29]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()
# Then in a new terminal: "tensorboard --logdir ." and go to the given link

This will produce an event file in the current directory with a name in the following format: events.out.tfevents.{timestamp}.{hostname}

### Session

To evaluate tensors, instantiate a tf.Session object, informally known as a session. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a tf.Graph is like a .py file, a tf.Session is like the python executable.

In [4]:
sess = tf.Session()
print(sess.run(total))

7.0


In [5]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


During a call to tf.Session.run any tf.Tensor only has a single value. For example, the following code calls tf.random_uniform to produce a tf.Tensor that generates a random 3-element vector (with values in [0,1)):

In [6]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.43155742 0.91663027 0.84837306]
[0.7344972  0.3213793  0.01072288]
(array([1.4357963, 1.9851023, 1.8954157], dtype=float32), array([2.4357963, 2.9851022, 2.8954158], dtype=float32))


Some TensorFlow functions return tf.Operations instead of tf.Tensors. The result of calling run on an Operation is None. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the initialization, and training ops demonstrated later.

### Feeding

As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A **placeholder** is a promise to provide a value later, like a function argument.

In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [8]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


### Datasets

Placeholders work for simple experiments, but tf.data are the preferred method of streaming data into a model.

To get a runnable tf.Tensor from a Dataset you must first convert it to a tf.data.Iterator, and then call the Iterator's tf.data.Iterator.get_next method.

The simplest way to create an Iterator is with the tf.data.Dataset.make_one_shot_iterator method. For example, in the following code the next_item tensor will return a row from the my_data array on each run call:

In [9]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,]]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [10]:
print(next_item)

Tensor("IteratorGetNext:0", shape=(2,), dtype=int32)


In [11]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


If the Dataset depends on stateful operations you may need to initialize the iterator before using it, as shown below:

In [12]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[-1.2043598 -1.2721106  1.1086024]
[-0.41640556 -0.7980787   0.46971002]
[-0.8711718  0.8562956  1.8200469]
[-1.4086558  -0.9062103  -0.04862318]
[ 1.2611067e-03 -1.1580313e+00 -2.1033127e+00]
[ 2.000524   1.6607436 -0.3377262]
[ 0.21408448 -0.24826463 -0.23280552]
[ 0.02504385 -0.9716149  -0.75342774]
[-1.4244932   0.22228295  0.49156746]
[ 1.0232182 -0.8194435 -1.7353516]


### Layers

Layers package together both the variables and the operations that act on them. For example a densely-connected layer performs a weighted sum across all inputs for each output and applies an optional activation function. The connection weights and biases are managed by the layer object.

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

In [14]:
print(y)

Tensor("dense/BiasAdd:0", shape=(?, 1), dtype=float32)


### Initializing Layers


The layer contains variables that must be initialized before they can be used. While it is possible to initialize variables individually, you can easily initialize all the variables in a TensorFlow graph as follows:

In [15]:
init = tf.global_variables_initializer()
sess.run(init)

IMPORTANT: Also note that this global_variables_initializer only initializes variables that existed in the graph when the initializer was created. So the initializer should be one of the last things added during graph construction.

In [16]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[2.0347443]
 [3.8809867]]


### Layer Function shortcuts

For each layer class (like tf.layers.Dense) TensorFlow also supplies a shortcut function (like tf.layers.dense). The only difference is that the shortcut function versions create and run the layer in a single call. For example, the following code is equivalent to the earlier version:

In [17]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[0.40362215]
 [3.0912771 ]]


While convenient, this approach allows no access to the tf.layers.Layer object. This makes introspection and debugging more difficult, and layer reuse impossible.

### Feature columns

The easiest way to experiment with feature columns is using the tf.feature_column.input_layer function. This function only accepts dense columns as inputs, so to view the result of a categorical column you must wrap it in an tf.feature_column.indicator_column. For example:

In [18]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [19]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

In [20]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


### Training

Now that you're familiar with the basics of core TensorFlow, let's train a small regression model manually.

In [21]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [22]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

In [23]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-1.1867876]
 [-2.3735752]
 [-3.5603628]
 [-4.7471504]]


In [24]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))

2.19561


In [25]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [26]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
print(loss_value)

0.0649862


In [27]:
print(sess.run(y_pred))

[[-0.41039103]
 [-1.1988626 ]
 [-1.9873341 ]
 [-2.7758055 ]]


In [28]:
print(sess.run(loss))

0.06459768
